# CS342 Final Project
### Dayou Ren; Spencer Petty
### dr37269; sap3758

In this project, we will be building and training an autoencoder architecture to generate images of CS professor. We are basically asking this question: "what does a neural network believe a CS professor should look like?"

# Imports

In [1]:
import torchvision
import torch

from training import train_vae
from netclasses import VAEConv
from testfuncs import test_noise_generation, compare_outputs

# Training

In [2]:
nepochs_pre = 1
nepochs_main = 64
latent_dims = 16
hnodes = 64

linmodel = VAEConv(latent_dims,hnodes)

In [3]:
pretrainset = torchvision.datasets.ImageFolder('./lfwdatacolor', transform=torchvision.transforms.ToTensor())
pretrainloader = torch.utils.data.DataLoader(pretrainset, batch_size=64, shuffle=True)
train_vae(linmodel, pretrainloader, nepochs = nepochs_pre, regularize=False)
print(f'After {nepochs_pre}  epochs of pretraining')
compare_outputs(linmodel)

  0%|          | 0/1 [00:00<?, ?epoch/s]

  0%|          | 0/201 [00:00<?, ?batch/s]

In [ ]:
trainset  = torchvision.datasets.ImageFolder('./profdatacolor', transform = torchvision.transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle=True)
train_vae(linmodel, trainloader, nepochs = nepochs_main, inter_epoch=False, regularize=False)